In [ ]:
# VERVESTACKS GRID VISUALIZATION - FLOW-SCALED WITH SOLAR/WIND
import xlwings as xw
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from spatial_utils import bus_id_to_commodity

print("=== LOADING DATA ===")

# 1. Load Excel data
try:
    wb = xw.Book('_for network diag.xlsx')
    ws = wb.sheets[0]
    data = ws.used_range.options(pd.DataFrame, header=1, index=False).value
    wb.close()  # Close Excel workbook after reading data
    print(f"✅ Excel data loaded: {len(data)} rows")
except Exception as e:
    print(f"❌ Error loading Excel: {e}")
    exit()

# Close the Excel workbook and app to release resources
try:
    wb.close()
    if xw.apps:
        for app in xw.apps:
            if not app.books:
                app.quit()
except Exception as e:
    print(f"Warning: Could not close Excel objects cleanly: {e}")


# 2. Load CHE bus data
try:
    bus_data = pd.read_csv('1_grids/output/CHE/CHE_clustered_buses.csv')
    print(f"✅ Bus data loaded: {len(bus_data)} buses")
except Exception as e:
    print(f"❌ Error loading grid data: {e}")
    exit()

print("\n=== DETECTING COLUMNS ===")

# 3. Detect columns
def find_col(possible_names, columns):
    for name in possible_names:
        for col in columns:
            if str(col).strip().lower() == name:
                return col
    return None

colnames = [str(c).strip() for c in data.columns]
process_col = find_col(['process', 'proc', 'Process'], colnames)
commodity_col = find_col(['commodity', 'comm', 'Commodity'], colnames)
consumption_col = find_col(['consumption'], colnames)
production_col = find_col(['production'], colnames)

print(f"Process column: {process_col}")
print(f"Commodity column: {commodity_col}")
print(f"Consumption column: {consumption_col}")
print(f"Production column: {production_col}")

if not (process_col and commodity_col and (consumption_col or production_col)):
    print(f"❌ Missing required columns!")
    exit()

# 4. Clean data - use basic cleaning (just process + commodity required)
required_cols = [process_col, commodity_col]
data_clean = data.dropna(subset=required_cols)
print(f"✅ Using basic cleaning: {len(data_clean)} rows")

print("\n=== COMMODITY TO LOCATION MAPPING ===")

# 5. Create commodity to bus coordinate mapping
commodity_to_coords = {}

# Build commodity coordinates lookup using spatial utility
for _, bus_row in bus_data.iterrows():
    bus_id = bus_row['bus_id']
    coords = (bus_row['x'], bus_row['y'])
    
    # Transform bus_id to commodity name
    commodity_name = bus_id_to_commodity(bus_id, add_prefix=True)
    commodity_to_coords[commodity_name] = coords

print(f"Created {len(commodity_to_coords)} commodity->coordinate mappings")

# Debug: Check for (0,0) coordinates in the mapping
zero_coords = []
for comm, coords in commodity_to_coords.items():
    if coords == (0, 0) or coords[0] == 0 or coords[1] == 0:
        zero_coords.append((comm, coords))

if zero_coords:
    print(f"\n⚠️  Found {len(zero_coords)} commodities with suspicious coordinates:")
    for comm, coords in zero_coords[:10]:  # Show first 10
        print(f"  - {comm} → {coords}")

# Also check the original bus data for (0,0) coordinates
print(f"\nChecking original bus data for (0,0) coordinates:")
zero_buses = bus_data[(bus_data['x'] == 0) | (bus_data['y'] == 0)]
if len(zero_buses) > 0:
    print(f"Found {len(zero_buses)} buses with zero coordinates:")
    for _, bus in zero_buses.head(5).iterrows():
        bus_id = bus['bus_id']
        transformed = bus_id_to_commodity(bus_id, add_prefix=True)
        print(f"  - Bus {bus_id} → {transformed} at ({bus['x']}, {bus['y']})")
else:
    print("No buses found with (0,0) coordinates in original data")

print("\n=== PROCESS POSITIONING & INFEASIBILITY ANALYSIS ===")

# 6. Position processes and identify IMPNRGZ flows
process_positions = {}
transmission_lines = []
impnrgz_flows = []  # Track flows from IMPNRGZ sources

for _, row in data_clean.iterrows():
    proc = row[process_col]
    comm = row[commodity_col]
    prod_val = row.get(production_col, 0) if production_col else 0
    cons_val = row.get(consumption_col, 0) if consumption_col else 0
    
    # Check if commodity has geographic location
    if comm in commodity_to_coords:
        coords = commodity_to_coords[comm]
        
        if proc.startswith('g_'):
            # Transmission processes: collect coordinates and flow values for line drawing
            line_found = False
            for line_entry in transmission_lines:
                if line_entry['process'] == proc:
                    line_entry['all_coords'].append(coords)
                    line_entry['flow_values'].append(max(abs(prod_val), abs(cons_val)))
                    line_found = True
                    break
            
            if not line_found:
                new_line = {
                    'process': proc,
                    'all_coords': [coords],
                    'flow_values': [max(abs(prod_val), abs(cons_val))]
                }
                transmission_lines.append(new_line)
        
        else:
            # Regular processes: position as nodes (inherit location from commodity)
            process_key = f"{proc}_{comm}"  # Unique key for each process-commodity pair
            process_positions[process_key] = {
                'process': proc,
                'commodity': comm,
                'coords': coords,
                'production': prod_val,
                'consumption': cons_val
            }
            
            # Track IMPNRGZ flows - identify infeasibility sources
            if 'IMPNRGZ' in proc and prod_val > 0:
                # Debug the (0,0) issue
                if coords == (0, 0):
                    print(f"🚨 DEBUG: Found IMPNRGZ at (0,0): {proc} → {comm}")
                    print(f"    Checking if commodity {comm} exists in mapping...")
                    if comm in commodity_to_coords:
                        print(f"    ✅ Commodity found with coords: {commodity_to_coords[comm]}")
                    else:
                        print(f"    ❌ Commodity NOT found in mapping!")
                
                impnrgz_flows.append({
                    'source_process': proc,
                    'commodity': comm,
                    'coords': coords,
                    'flow': prod_val
                })

print(f"Positioned {len(process_positions)} process instances")
print(f"Found {len(transmission_lines)} transmission lines")
print(f"🚨 INFEASIBILITY ALERT: {len(impnrgz_flows)} IMPNRGZ flows detected!")

if impnrgz_flows:
    print("\nIMPNRGZ Sources (Infeasibility Points):")
    for flow in impnrgz_flows:
        coords = flow['coords']
        print(f"  - {flow['source_process']} → {flow['commodity']} (Flow: {flow['flow']:.1f}) at coords {coords}")
        
        # Flag suspicious coordinates
        if coords == (0, 0):
            print(f"    🚨 WARNING: {flow['source_process']} has (0,0) coordinates!")
        elif coords[0] == 0 or coords[1] == 0:
            print(f"    ⚠️  SUSPICIOUS: {flow['source_process']} has zero coordinate: {coords}")

# 7. Trace IMPNRGZ flows to demand nodes
print("\n=== TRACING INFEASIBILITY FLOWS ===")
infeasible_paths = []

if impnrgz_flows:
    # For each IMPNRGZ source, find what consumes from the same commodity
    for impnrgz_source in impnrgz_flows:
        source_commodity = impnrgz_source['commodity']
        source_coords = impnrgz_source['coords']
        source_flow = impnrgz_source['flow']
        
        # Find all processes that consume from this commodity
        consumers = []
        for proc_key, proc_info in process_positions.items():
            if (proc_info['commodity'] == source_commodity and 
                proc_info['consumption'] > 0):
                consumers.append(proc_info)
        
        # Check if any consumers are demand nodes
        for consumer in consumers:
            if consumer['process'].startswith('e_demand'):
                infeasible_paths.append({
                    'source': impnrgz_source['source_process'],
                    'source_coords': source_coords,
                    'target': consumer['process'],
                    'target_coords': consumer['coords'],
                    'commodity': source_commodity,
                    'flow': min(source_flow, consumer['consumption'])  # Actual flow
                })
                print(f"🔴 INFEASIBLE PATH: {impnrgz_source['source_process']} → {consumer['process']} (Flow: {min(source_flow, consumer['consumption']):.1f})")

print(f"Found {len(infeasible_paths)} direct infeasible paths to demand nodes")

print("\n=== CREATING VISUALIZATION ===")

# 7. Create the visualization
fig, ax = plt.subplots(1, 1, figsize=(16, 12))

# Calculate flow-based line thickness scaling
all_flows = []
for line in transmission_lines:
    all_flows.extend(line['flow_values'])
max_flow = max(all_flows) if all_flows else 1
min_flow = min([f for f in all_flows if f > 0]) if all_flows else 1

# Draw transmission lines (g_ processes) with flow-based thickness
transmission_labels = []  # Store labels for callout positioning
for line in transmission_lines:
    proc = line['process']
    coords_list = line['all_coords']
    flow_values = line['flow_values']
    
    # Calculate average flow for this transmission line
    avg_flow = sum(flow_values) / len(flow_values) if flow_values else 1
    
    # Scale line thickness based on flow (1-8 pixel range)
    if max_flow > min_flow:
        thickness = 1 + 7 * (avg_flow - min_flow) / (max_flow - min_flow)
    else:
        thickness = 2
    thickness = max(1, min(8, thickness))  # Clamp between 1-8
    
    # Draw lines between all coordinate pairs for this transmission process
    for i, coord1 in enumerate(coords_list):
        for j, coord2 in enumerate(coords_list):
            if i < j:  # Avoid duplicate lines and self-connections
                ax.plot([coord1[0], coord2[0]], 
                       [coord1[1], coord2[1]], 
                       color='#888888', linewidth=thickness, alpha=0.8, zorder=1)
                
                # Store label position for callout (midpoint of first line only)
                if len(transmission_labels) < 15:  # Limit labels to avoid clutter
                    mid_x = (coord1[0] + coord2[0]) / 2
                    mid_y = (coord1[1] + coord2[1]) / 2
                    transmission_labels.append({
                        'text': proc.split('-')[0],
                        'line_pos': (mid_x, mid_y),
                        'flow': avg_flow
                    })

# Categorize and draw process nodes with flow-based sizing
ep_processes = []
solar_wind_processes = []  # Renamed from distr_processes
demand_processes = []
other_processes = []

# Calculate min/max flows for node scaling
all_node_flows = []
for proc_key, proc_info in process_positions.items():
    flow = max(abs(proc_info['production']), abs(proc_info['consumption']))
    if flow > 0:
        all_node_flows.append(flow)

max_node_flow = max(all_node_flows) if all_node_flows else 1
min_node_flow = min(all_node_flows) if all_node_flows else 1

for proc_key, proc_info in process_positions.items():
    proc = proc_info['process']
    coords = proc_info['coords']
    flow = max(abs(proc_info['production']), abs(proc_info['consumption']))
    
    # Calculate node size based on flow (50-400 pixel range)
    if max_node_flow > min_node_flow and flow > 0:
        size = 50 + 350 * (flow - min_node_flow) / (max_node_flow - min_node_flow)
    else:
        size = 100
    size = max(50, min(400, size))  # Clamp between 50-400
    
    if proc.startswith('ep_'):
        ep_processes.append((coords, proc, size))
    elif proc.startswith('distr'):
        # Determine if it's solar or wind based on process name
        if 'solar' in proc.lower() or 'pv' in proc.lower():
            label = proc.replace('distr', 'solar')
        elif 'wind' in proc.lower():
            label = proc.replace('distr', 'wind')
        else:
            label = proc.replace('distr', 'solar/wind')
        solar_wind_processes.append((coords, label, size))
    elif proc.startswith('e_demand'):
        demand_processes.append((coords, proc, size))
    else:
        other_processes.append((coords, proc, size))

# Draw different process types with selective labeling for focused debugging
def draw_processes_with_selective_labels(process_list, color, label_prefix, focus_commodity=None):
    if process_list:
        # Extract coordinates, labels, and sizes
        coords_list = [p[0] for p in process_list]
        labels_list = [p[1] for p in process_list]
        sizes_list = [p[2] for p in process_list]
        
        x_coords = [c[0] for c in coords_list]
        y_coords = [c[1] for c in coords_list]
        
        # Draw nodes with individual sizes
        ax.scatter(x_coords, y_coords, c=color, s=sizes_list, alpha=0.9, 
                  marker='s', edgecolors='white', linewidths=2, zorder=3)
        
                         # Collect related processes for external labeling
        if focus_commodity:
            related_processes = []
            for i, ((x, y), label, size) in enumerate(process_list):
                # Check if this process is related to the focus commodity
                is_related = False
                for proc_key, proc_info in process_positions.items():
                    if (proc_info['coords'] == (x, y) and 
                        proc_info['commodity'] == focus_commodity):
                        is_related = True
                        break
                
                if is_related:
                    related_processes.append({
                        'coords': (x, y),
                        'label': label,
                        'size': size,
                        'color': color
                    })
            
            return related_processes
        return []

# Determine focus commodity if we have infeasible paths
focus_commodity = None
if infeasible_paths:
    biggest_path = max(infeasible_paths, key=lambda x: x['flow'])
    focus_commodity = biggest_path['commodity']

# Draw process types and collect related processes for external labeling
all_related_processes = []
related_ep = draw_processes_with_selective_labels(ep_processes, '#2E8B57', 'ep_', focus_commodity) or []
related_solar = draw_processes_with_selective_labels(solar_wind_processes, '#FFB347', 'solar/wind', focus_commodity) or []
related_demand = draw_processes_with_selective_labels(demand_processes, '#DC143C', 'e_demand', focus_commodity) or []
related_other = draw_processes_with_selective_labels(other_processes, '#2E5BBA', 'other', focus_commodity) or []

all_related_processes.extend(related_ep)
all_related_processes.extend(related_solar)
all_related_processes.extend(related_demand)
all_related_processes.extend(related_other)

# Add external labels with leader lines
if all_related_processes:
    # Get plot boundaries
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Position labels outside the network area (right side)
    label_x = xlim[1] + 0.05 * (xlim[1] - xlim[0])  # 5% beyond right edge
    
    # Distribute labels vertically
    y_spacing = (ylim[1] - ylim[0]) / (len(all_related_processes) + 1)
    
    for i, proc_info in enumerate(all_related_processes):
        coords = proc_info['coords']
        label = proc_info['label']
        color = proc_info['color']
        
        # Position for this external label
        label_y = ylim[1] - (i + 1) * y_spacing
        
        # Draw leader line from process to external label
        ax.plot([coords[0], label_x], [coords[1], label_y], 
               color='red', linewidth=1, alpha=0.7, linestyle='-', zorder=14)
        
        # Add external label
        ax.text(label_x, label_y, label[:15], fontsize=9, ha='left', va='center',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.9, edgecolor='red'),
               fontweight='bold', zorder=15)

# 🚨 FOCUS ON BIGGEST INFEASIBILITY - Trace the largest flow path
if infeasible_paths:
    # Find the biggest infeasible flow
    biggest_path = max(infeasible_paths, key=lambda x: x['flow'])
    biggest_source_commodity = biggest_path['commodity']
    biggest_flow = biggest_path['flow']
    
    print(f"🔴 FOCUSING ON BIGGEST INFEASIBILITY:")
    print(f"   Commodity: {biggest_source_commodity}")
    print(f"   Flow: {biggest_flow:.1f} TWh")
    print(f"   Path: {biggest_path['source']} → {biggest_path['target']}")
    
    # Draw ONLY the biggest infeasible path
    source_coords = biggest_path['source_coords']
    target_coords = biggest_path['target_coords']
    
    # Draw thick RED line for the biggest infeasible flow
    ax.plot([source_coords[0], target_coords[0]], 
           [source_coords[1], target_coords[1]], 
           color='red', linewidth=8, alpha=0.9, zorder=10,
           linestyle='--')  # Dashed red line
    
         # No label on the line itself - keep it clean to see underlying processes
    
    # Collect all processes related to this commodity for focused labeling
    related_processes = []
    for proc_key, proc_info in process_positions.items():
        if proc_info['commodity'] == biggest_source_commodity:
            related_processes.append(proc_info)
    
    print(f"   Found {len(related_processes)} processes related to {biggest_source_commodity}")
    
    # Store the biggest path info for later use
    focus_commodity = biggest_source_commodity
    focus_source_coords = source_coords
    focus_target_coords = target_coords
    focus_flow = biggest_flow

# Skip transmission line callouts to keep the network area clean

print(f"Process breakdown:")
print(f"  - Generators (ep_): {len(ep_processes)}")
print(f"  - Solar/Wind (distr): {len(solar_wind_processes)}")
print(f"  - Demand (e_demand): {len(demand_processes)}")
print(f"  - Transmission lines (g_): {len(transmission_lines)}")
print(f"  - Other processes: {len(other_processes)}")

# Styling with infeasibility focus
if infeasible_paths:
    title = f'VerveStacks Grid - INFEASIBILITY DEBUGGING\n🚨 {len(infeasible_paths)} Infeasible Paths Found (IMPNRGZ → Demand)'
    title_color = 'red'
else:
    title = 'VerveStacks Grid Architecture - Flow-Scaled\nNo Infeasibilities Detected ✅'
    title_color = 'black'

ax.set_title(title, fontsize=16, fontweight='bold', pad=20, color=title_color)

# Create legend with infeasibility indicators
legend_elements = [
    Rectangle((0,0), 1, 1, facecolor='#2E8B57', edgecolor='white', label='Generators (ep_)'),
    Rectangle((0,0), 1, 1, facecolor='#FFB347', edgecolor='white', label='Solar/Wind Collection'),
    Rectangle((0,0), 1, 1, facecolor='#DC143C', edgecolor='white', label='Demand (e_demand)'),
    plt.Line2D([0], [0], color='#888888', linewidth=3, label='Transmission Lines (g_)')
]

# Add infeasibility legend items if present
if infeasible_paths:
    legend_elements.extend([
        plt.Line2D([0], [0], color='red', linewidth=4, linestyle='--', label='🚨 Infeasible Flows (TWh)'),
        Rectangle((0,0), 1, 1, facecolor='red', edgecolor='darkred', label='Infeasible Commodities')
    ])

ax.legend(handles=legend_elements, loc='upper right', fontsize=11, 
          title='VerveStacks Debugging\n(Node/Line size ∝ Flow)', title_fontsize=12)

ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.grid(True, alpha=0.3)

# Add focused infeasibility info on the left edge
if infeasible_paths:
    # Get plot boundaries
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Position for the info box (left edge)
    list_x = xlim[0] + 0.02 * (xlim[1] - xlim[0])  # 2% from left edge
    
    # Show only the biggest infeasibility
    biggest_flow = max(impnrgz_flows, key=lambda x: x['flow'])
    commodity = biggest_flow['commodity']
    flow_val = biggest_flow['flow']
    actual_coords = biggest_flow['coords']
    
    # Position for the focused info
    info_y = ylim[1] - 0.15 * (ylim[1] - ylim[0])
    
    # Draw red box with the biggest infeasibility info (no title)
    ax.text(list_x, info_y, f"{commodity}\n{flow_val:.1f} TWh", 
           fontsize=10, fontweight='bold', ha='left', va='center',
           bbox=dict(boxstyle='round,pad=0.4', facecolor='red', alpha=0.9, edgecolor='darkred'),
           color='white', zorder=15)
    
    # Draw connecting line from info to actual location
    ax.plot([list_x + 0.08, actual_coords[0]], [info_y, actual_coords[1]], 
           color='red', linewidth=2, alpha=0.8, linestyle=':', zorder=14)

plt.tight_layout()
plt.show()

print(f"\n✅ VerveStacks visualization complete!")
print(f"   - Commodities: Invisible (location carriers only)")
print(f"   - Processes: {len(process_positions)} positioned instances")
print(f"   - Transmission: {len(transmission_lines)} lines between coordinates")
